In [1]:
from google.colab import files
uploaded = files.upload()

Saving MasterResearchFunction.py to MasterResearchFunction.py


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install japanize_matplotlib bottleneck tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 32.0 MB/s eta 0:00:00
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=e7a6d4e4bd5c45b9e6af8fad2de580789e708e2fccdf725857364ba6c68f36cb
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


In [4]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

In [5]:
#Apple Watchのモーションデータの読み込み
#a_yuuma_motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
#b_sakamoto_motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
#c_watabe_motion_data = mr.process_apple_watch_csv("datasets/watabe/20240605/watabe_motion.csv")
#d_nakazawa_motion_data = mr.process_apple_watch_csv("datasets/nakazawa/20240606/nakazawa_motion.csv")
#e_okede_motion_data = mr.process_apple_watch_csv("datasets/okeda/20240607/okeda_motion.csv")
a_yuuma_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_motion.cs")
b_sakamoto_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_motion.csv")
c_watabe_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_motion.csv")
d_nakazawa_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_motion.csv")
e_okede_motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_motion.csv")

In [ ]:
#Tobiiのモーションデータの読み込み
#a_yuuma_eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")
#b_sakamoto_eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")
#c_watabe_eye_data = mr.process_tobii_csv("datasets/watabe/20240605/watabe_eye.csv")
#d_nakazawa_eye_data = mr.process_tobii_csv("datasets/nakazawa/20240606/nakazawa_eye.csv")
#e_okede_eye_data = mr.process_tobii_csv("datasets/okeda/20240607/okeda_eye.csv")
a_yuuma_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/yuuma/20240604/yuuma_eye.cs")
b_sakamoto_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/sakamoto/20240604/sakamoto_eye.csv")
c_watabe_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/watabe/20240605/watabe_eye.csv")
d_nakazawa_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/nakazawa/20240606/nakazawa_eye.csv")
e_okede_eye_data = mr.process_tobii_csv("/content/drive/MyDrive/datasets/okeda/20240607/okeda_eye.csv")

In [7]:
#a_yuumaくんの教師データ読み込み
# a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/check")
# a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/circle")
# a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/cross")
# a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("datasets/yuuma/train_gesture/tri")
a_yuuma_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/check")
a_yuuma_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/circle")
a_yuuma_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/cross")
a_yuuma_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/yuuma/train_gesture/tri")

In [ ]:
#b_sakamotoくんの教師データ読み込み
# b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/check")
# b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/circle")
# b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/cross")
# b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/tri")
b_sakamoto_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/check")
b_sakamoto_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/circle")
b_sakamoto_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/cross")
b_sakamoto_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/sakamoto/train_gesture/tri")

In [ ]:
#c_watabeくんの教師データ読み込み
# c_watabe_check = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/check")
# c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/circle")
# c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/cross")
# c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("datasets/watabe/train_gesture/tri")
c_watabe_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/check")
c_watabe_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/circle")
c_watabe_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/cross")
c_watabe_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/watabe/train_gesture/tri")

In [ ]:
#d_nakazawaくんの教師データ読み込み
# d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/check")
# d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/circle")
# d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/cross")
# d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("datasets/nakazawa/train_gesture/tri")
d_nakazawa_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/check")
d_nakazawa_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/circle")
d_nakazawa_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/cross")
d_nakazawa_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/nakazawa/train_gesture/tri")

In [ ]:
#e_okedaくんの教師データ読み込み
# e_okeda_check = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/check")
# e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/circle")
# e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/cross")
# e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("datasets/okeda/train_gesture/tri")
e_okeda_check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/check")
e_okeda_circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/circle")
e_okeda_cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/cross")
e_okeda_tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/okeda/train_gesture/tri")

In [10]:
# Th = 10
# segx, segy, segz = [], [], []

In [11]:
for i in range(len(circle)):
  segx.append(mr.spring_ogawa(motion_data['AccelerationX'], circle[i]['AccelerationX'], Th))
  segy.append(mr.spring_ogawa(motion_data['AccelerationY'], circle[i]['AccelerationY'], Th))
  segz.append(mr.spring_ogawa(motion_data['AccelerationZ'], circle[i]['AccelerationZ'], Th))

ストリーミング出力は最後の 5000 行に切り捨てられました。
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
***

In [21]:
overlap = []
for i in range(len(segx)):
  overlap.append(mr.combine_and_find_overlapping_segments(segx[i], segy[i], segz[i]))

In [325]:
#短くなるやつ
def combine_all_overlaps(overlap):
    # Combine all segments from multiple lists into one list
    combined_segments = []
    for segments in overlap:
        combined_segments.extend(segments)

    # Sort combined segments
    combined_segments.sort()

    # Find and merge overlapping segments
    final_segments = []
    current_overlap = None

    for start, end in combined_segments:
        if current_overlap is None:
            current_overlap = (start, end)
        else:
            current_start, current_end = current_overlap
            if start <= current_end:
                current_overlap = (current_start, max(current_end, end))
            else:
                final_segments.append(current_overlap)
                current_overlap = (start, end)

    if current_overlap is not None:
        final_segments.append(current_overlap)

    return final_segments

In [299]:
#長くなるやつ
def combine_all_overlaps(overlap):
    # Combine all segments from multiple lists into one list
    combined_segments = []
    for segments in overlap:
        combined_segments.extend(segments)

    # Sort combined segments
    combined_segments.sort()

    # Find and manage overlapping segments
    final_segments = []
    current_segments = []

    for start, end in combined_segments:
        if not current_segments:
            current_segments.append((start, end))
        else:
            last_start, last_end = current_segments[-1]
            if start <= last_end:
                current_segments.append((start, end))
            else:
                final_segments.append(current_segments)
                current_segments = [(start, end)]

    if current_segments:
        final_segments.append(current_segments)

    return final_segments

In [334]:
#適度なやつ
def combine_all_overlaps_with_limit(overlap, max_segment_length):
    """
    Combine overlapping segments, ensuring no segment exceeds max_segment_length.
    """
    # Combine all segments from multiple lists into one list
    combined_segments = []
    for segments in overlap:
        combined_segments.extend(segments)

    # Sort combined segments
    combined_segments.sort()

    # Find and merge overlapping segments with length limitation
    final_segments = []
    current_overlap = None

    for start, end in combined_segments:
        if current_overlap is None:
            current_overlap = (start, end)
        else:
            current_start, current_end = current_overlap
            if start <= current_end:
                if (max(current_end, end) - current_start) <= max_segment_length:
                    current_overlap = (current_start, max(current_end, end))
                else:
                    final_segments.append(current_overlap)
                    current_overlap = (start, end)
            else:
                final_segments.append(current_overlap)
                current_overlap = (start, end)

    if current_overlap is not None:
        final_segments.append(current_overlap)

    return final_segments

In [335]:
# Max segment length in time units (e.g., seconds)
max_segment_length = 250  # Adjust this value as needed

final_segments = combine_all_overlaps_with_limit(overlap, max_segment_length)
print(final_segments)

[(119, 241), (305, 359), (452, 521), (604, 665), (890, 963), (1394, 1553), (1603, 1743), (1745, 1861), (2093, 2279), (2424, 2517), (3429, 3512), (3845, 3978), (3977, 4100), (4601, 4753), (5360, 5524), (5526, 5605), (6975, 7100), (7394, 7508), (8090, 8157), (8559, 8598), (8948, 9146), (9489, 9665), (9694, 9919), (9748, 10011), (9921, 10046), (9933, 10183), (9936, 10495), (9937, 10048), (10023, 10232), (10090, 10419), (10277, 10462), (10463, 10643), (10670, 10872), (11002, 11112), (11315, 11472), (11569, 11789), (11822, 12071), (11868, 12162), (11869, 12011), (11935, 12253), (11945, 12102), (11945, 12253), (12005, 12253), (12125, 12376), (12131, 12376), (12315, 12438), (12391, 12634), (12436, 12675), (12658, 12945), (12681, 12934), (12693, 12935), (12693, 12945), (12700, 12938), (12955, 13142), (13131, 13565), (13135, 13237), (13135, 13439), (13158, 13439), (13200, 13438), (13322, 13561), (13735, 13914), (13913, 14036), (13928, 14235), (14030, 14271), (14210, 14388), (14280, 14483), (145

In [341]:
motion_data['Timestamp'][10046]

Timestamp('2024-06-04 18:35:00.396894976')

In [332]:
final_segments[21]

(9694, 10643)

In [88]:
circle10 = overlap

In [327]:
circle10 = final_segments

In [328]:
final_segments

[(119, 241),
 (305, 359),
 (452, 521),
 (604, 665),
 (890, 963),
 (1394, 1553),
 (1603, 1743),
 (1745, 1861),
 (2093, 2279),
 (2424, 2517),
 (3429, 3512),
 (3845, 4100),
 (4601, 4753),
 (5360, 5524),
 (5526, 5605),
 (6975, 7100),
 (7394, 7508),
 (8090, 8157),
 (8559, 8598),
 (8948, 9146),
 (9489, 9665),
 (9694, 10643),
 (10670, 10872),
 (11002, 11112),
 (11315, 11472),
 (11569, 11789),
 (11822, 12945),
 (12955, 13565),
 (13735, 14483),
 (14520, 15753),
 (15756, 18052),
 (18057, 23222),
 (23223, 26303),
 (26354, 26593),
 (26598, 28766),
 (28767, 29267),
 (29278, 29646),
 (29698, 29946),
 (29947, 31073),
 (31075, 31195),
 (31261, 31374),
 (31438, 34489),
 (34518, 35012),
 (35013, 36794),
 (36795, 37307),
 (37325, 39729),
 (39748, 40353),
 (40375, 40850),
 (40959, 41061),
 (41173, 41284),
 (41603, 41731),
 (42276, 42332),
 (43845, 43901),
 (44815, 44856),
 (46238, 46299),
 (48311, 48409),
 (51270, 51326),
 (52760, 52913),
 (58741, 58897),
 (59359, 59454),
 (61187, 61246),
 (63999, 64305),

In [91]:
results = []
for i in range(len(circle10)):
  temp = []
  for start, end in circle10[i]:
    temp.append([motion_data['Timestamp'][start], motion_data['Timestamp'][end]])
  results.append(temp)

In [330]:
# circle10 の例
circle10 = final_segments

# 結果を格納するリスト
results = []
for start, end in circle10:
    a = [motion_data['Timestamp'][start], motion_data['Timestamp'][end]]
    results.append(a)

# 経過時間を計算して格納するリスト
elapsed_times = []

# 各区間の経過時間を計算
for start_time, end_time in results:
    # 経過時間を計算
    elapsed_time = (end_time - start_time).total_seconds()

    # 経過時間をリストに追加
    elapsed_times.append(elapsed_time)

# 経過時間のリストを表示
print("Elapsed times (seconds):", elapsed_times)

Elapsed times (seconds): [2.438843, 1.079584, 1.379146, 1.219371, 1.459326, 3.17859, 2.798654, 2.3189640000000002, 3.718157, 1.8590659999999999, 1.659019, 5.097376, 3.038715, 3.27833, 1.579122, 2.498626, 2.279027, 1.339262, 0.779738, 3.958065, 3.5181, 18.970365, 4.037917, 2.198854, 3.138364, 4.39788, 22.447972, 12.193968, 14.952172000000001, 24.647131, 45.895483, 103.244017, 61.566252, 4.7774529999999995, 43.3359, 9.994547, 7.355767, 4.957291, 22.507499, 2.398793, 2.258686, 60.985211, 9.874347, 35.599684, 10.234369, 48.052423, 12.093106, 9.494675, 2.037662, 2.218422, 2.558481, 1.119473, 1.119343, 0.819511, 1.219257, 1.9588619999999999, 1.119267, 3.058333, 3.118164, 1.8987880000000001, 1.1793770000000001, 6.1166, 1.079396, 2.478738, 2.178782, 2.978177, 3.558113, 5.336857, 5.6167359999999995, 2.7984809999999998, 2.43868, 3.178408, 4.897016, 2.038853, 2.318749, 2.158604, 1.518977, 5.656926, 2.918526, 1.6791939999999999, 1.339313, 2.078817, 2.478686, 1.239258, 1.239302, 1.119396, 0.919525,

In [333]:
elapsed_times[21]

18.970365

In [320]:
results

[[[Timestamp('2024-06-04 18:31:45.675748096'),
   Timestamp('2024-06-04 18:31:46.755332864')],
  [Timestamp('2024-06-04 18:31:48.634483968'),
   Timestamp('2024-06-04 18:31:49.993710080')],
  [Timestamp('2024-06-04 18:31:57.370010880'),
   Timestamp('2024-06-04 18:31:58.249645824')],
  [Timestamp('2024-06-04 18:32:57.479557120'),
   Timestamp('2024-06-04 18:32:59.039154944')],
  [Timestamp('2024-06-04 18:33:26.744775168'),
   Timestamp('2024-06-04 18:33:30.003104')],
  [Timestamp('2024-06-04 18:33:59.747952896'),
   Timestamp('2024-06-04 18:34:01.406790144')],
  [Timestamp('2024-06-04 18:34:07.464108032'),
   Timestamp('2024-06-04 18:34:09.663123968')],
  [Timestamp('2024-06-04 18:34:39.347776'),
   Timestamp('2024-06-04 18:34:42.306269952')],
  [Timestamp('2024-06-04 18:34:49.262653952'),
   Timestamp('2024-06-04 18:34:52.740810752')],
  [Timestamp('2024-06-04 18:34:53.860209920'),
   Timestamp('2024-06-04 18:34:56.319174144')],
  [Timestamp('2024-06-04 18:34:58.198156032'),
   Timest

In [95]:
# 経過時間を計算して格納するリスト
elapsed_times = []

# 各区間の経過時間を計算
for i in range(len(results)):
  temp = []
  for start_time, end_time in results[i]:
    # 経過時間を計算
    temp.append((end_time - start_time).total_seconds())
  # 経過時間をリストに追加
  elapsed_times.append(temp)

# 経過時間のリストを表示
print("Elapsed times (seconds):", elapsed_times)

Elapsed times (seconds): [[1.079584, 1.359226, 0.879634, 1.5595970000000001, 3.258328, 1.6588370000000001, 2.199015, 2.958493, 3.4781560000000002, 2.458964, 11.174132, 3.138366, 2.618653, 1.179404, 4.777521, 4.258335, 2.358693, 2.038882, 4.777493, 2.398597, 2.458791, 1.5993439999999999, 3.557985, 1.039503, 2.578609, 2.438652, 2.438673, 3.018394, 2.438711, 1.099273, 2.138917, 2.2987349999999998, 3.418327, 1.39924, 2.359056, 3.418157, 4.237792, 2.399043, 4.177716, 1.819022, 2.038797, 2.258755, 2.698664, 4.857417, 4.577383, 2.41865, 2.418465, 5.89687, 2.4587149999999998, 3.478162, 2.498609, 7.07619, 2.398824, 3.138189, 3.598007, 2.538846, 2.238776, 2.138763, 3.258091, 7.595852, 4.53746, 1.719169, 0.97956, 2.178801, 1.878933, 1.179468, 2.51866, 2.218829, 2.358747, 4.357635, 2.498833, 2.71835, 2.318727, 2.378593, 2.818479, 2.138817, 3.03827, 2.698691, 3.078168, 3.058296, 2.378578, 6.336525, 4.757405, 3.338261, 2.398575, 2.318863, 2.198818, 2.258861, 3.2783189999999998, 1.559122, 5.616851000

In [97]:
len(elapsed_times[0])

169

In [132]:
# 条件に従ってインデックスの組を削除
filtered_circle10 = []
filtered_results = []
filtered_elapsed_times = []

for i in range(len(elapsed_times)):
    temp_circle10 = []
    temp_results = []
    temp_elapsed_times = []
    for j in range(len(elapsed_times[i])):
        elapsed_time = elapsed_times[i][j]
        if elapsed_time > 1.5 and elapsed_time < 6:
            temp_circle10.append(circle10[i][j])
            temp_results.append(results[i][j])
            temp_elapsed_times.append(elapsed_time)
    if temp_circle10:  # フィルタリング後にデータが存在する場合のみ追加
        filtered_circle10.append(temp_circle10)
        filtered_results.append(temp_results)
        filtered_elapsed_times.append(temp_elapsed_times)

In [133]:
# フィルタリング後のリストを表示
print("Filtered circle10:", filtered_circle10)
print("Filtered results:", filtered_results)
print("Filtered elapsed times (seconds):", filtered_elapsed_times)

Filtered circle10: [[(3897, 3975), (5361, 5524), (7012, 7095), (7398, 7508), (8993, 9141), (9489, 9663), (9719, 9842), (11945, 12102), (12122, 12253), (12436, 12675), (12719, 12932), (13011, 13129), (13135, 13237), (13322, 13561), (13791, 13911), (13913, 14036), (14126, 14206), (14210, 14388), (14757, 14886), (14949, 15071), (15118, 15240), (15380, 15531), (15631, 15753), (15833, 15940), (16006, 16121), (16207, 16378), (16654, 16772), (16828, 16999), (17003, 17215), (17270, 17390), (17392, 17601), (17660, 17751), (17786, 17888), (17939, 18052), (18062, 18197), (18219, 18462), (18516, 18745), (18752, 18873), (18874, 18995), (19356, 19479), (19483, 19657), (19660, 19785), (20279, 20399), (20487, 20644), (20646, 20826), (20870, 20997), (21127, 21239), (21244, 21351), (21475, 21638), (22031, 22258), (22350, 22436), (22499, 22608), (22623, 22717), (22861, 22987), (23105, 23216), (23277, 23395), (23422, 23640), (23742, 23867), (23903, 24039), (24158, 24274), (24330, 24449), (24607, 24748), (

In [225]:
filtered_results

[[[Timestamp('2024-06-04 18:32:57.479557120'),
   Timestamp('2024-06-04 18:32:59.039154944')],
  [Timestamp('2024-06-04 18:33:26.744775168'),
   Timestamp('2024-06-04 18:33:30.003104')],
  [Timestamp('2024-06-04 18:33:59.747952896'),
   Timestamp('2024-06-04 18:34:01.406790144')],
  [Timestamp('2024-06-04 18:34:07.464108032'),
   Timestamp('2024-06-04 18:34:09.663123968')],
  [Timestamp('2024-06-04 18:34:39.347776'),
   Timestamp('2024-06-04 18:34:42.306269952')],
  [Timestamp('2024-06-04 18:34:49.262653952'),
   Timestamp('2024-06-04 18:34:52.740810752')],
  [Timestamp('2024-06-04 18:34:53.860209920'),
   Timestamp('2024-06-04 18:34:56.319174144')],
  [Timestamp('2024-06-04 18:35:38.357222912'),
   Timestamp('2024-06-04 18:35:41.495589888')],
  [Timestamp('2024-06-04 18:35:41.895377152'),
   Timestamp('2024-06-04 18:35:44.514030848')],
  [Timestamp('2024-06-04 18:35:48.172148992'),
   Timestamp('2024-06-04 18:35:52.949670912')],
  [Timestamp('2024-06-04 18:35:53.829151232'),
   Timest

In [135]:
# filtered_results の範囲内の eye_data['Timestamp'] を抽出する関数
def extract_eye_data_within_intervals(filtered_results, eye_data):
    extracted_eye_data = []

    for segment in filtered_results:
        segment_data = []
        for start_time, end_time in segment:
            # タイムスタンプが範囲内のデータを抽出
            mask = (eye_data['Timestamp'] >= start_time) & (eye_data['Timestamp'] <= end_time)
            segment_data.append(eye_data[mask])
        extracted_eye_data.append(segment_data)

    return extracted_eye_data

In [136]:
# タイムスタンプの範囲内のデータを抽出
extracted_eye_data = extract_eye_data_within_intervals(filtered_results, eye_data)

In [247]:
# 分散を計算し、閾値を上回る区間を削除する関数
def filter_by_variance(extracted_eye_data, filtered_results, threshold):
    filtered_extracted_eye_data = []
    filtered_filtered_results = []

    for i in range(len(extracted_eye_data)):
        temp_extracted_eye_data = []
        temp_filtered_results = []
        for j in range(len(extracted_eye_data[i])):
            gaze_x_var = extracted_eye_data[i][j]['Pupil diameter left'].diff().var()
            gaze_y_var = extracted_eye_data[i][j]['Pupil diameter right'].diff().var()

            if gaze_x_var < threshold and gaze_y_var < threshold:
                temp_extracted_eye_data.append(extracted_eye_data[i][j])
                temp_filtered_results.append(filtered_results[i][j])

        if temp_extracted_eye_data:  # フィルタリング後にデータが存在する場合のみ追加
            filtered_extracted_eye_data.append(temp_extracted_eye_data)
            filtered_filtered_results.append(temp_filtered_results)

    return filtered_extracted_eye_data, filtered_filtered_results

In [251]:
Th = 0.0045

In [252]:
# 分散を計算し、閾値を上回る区間を削除
filtered_extracted_eye_data, filtered_filtered_results = filter_by_variance(extracted_eye_data, filtered_results, Th)

In [249]:
len(filtered_filtered_results[0])

10

In [360]:
start_time = '2024-06-04 19:21:52'
end_time = '2024-06-04 19:21:55'

In [361]:
filtered_data = eye_data[(eye_data['Timestamp'] >= start_time) & (eye_data['Timestamp'] <= end_time)]

In [363]:
filtered_data['Gaze point X'].diff().var()

1284.199925037481

In [365]:
filtered_data['Pupil diameter right'].diff().var()

0.015155823010323003

In [314]:
filtered_data.columns

Index(['Sensor', 'Participant name', 'Event', 'Event value', 'Gaze point X',
       'Gaze point Y', 'Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z',
       'Gaze direction left X', 'Gaze direction left Y',
       'Gaze direction left Z', 'Gaze direction right X',
       'Gaze direction right Y', 'Gaze direction right Z',
       'Pupil position left X', 'Pupil position left Y',
       'Pupil position left Z', 'Pupil position right X',
       'Pupil position right Y', 'Pupil position right Z',
       'Pupil diameter left', 'Pupil diameter right', 'Validity left',
       'Validity right', 'Eye movement type', 'Gaze event duration',
       'Eye movement type index', 'Fixation point X', 'Fixation point Y',
       'Ungrouped', 'Timestamp'],
      dtype='object')

In [254]:
filtered_filtered_results

[[[Timestamp('2024-06-04 18:32:57.479557120'),
   Timestamp('2024-06-04 18:32:59.039154944')],
  [Timestamp('2024-06-04 18:33:59.747952896'),
   Timestamp('2024-06-04 18:34:01.406790144')],
  [Timestamp('2024-06-04 18:37:49.707787008'),
   Timestamp('2024-06-04 18:37:54.285170176')],
  [Timestamp('2024-06-04 18:40:52.987462912'),
   Timestamp('2024-06-04 18:40:54.546585088')],
  [Timestamp('2024-06-04 18:44:45.635715840'),
   Timestamp('2024-06-04 18:44:47.934376960')],
  [Timestamp('2024-06-04 19:07:00.671313920'),
   Timestamp('2024-06-04 19:07:02.590368')],
  [Timestamp('2024-06-04 19:17:46.235107072'),
   Timestamp('2024-06-04 19:17:48.553856768')],
  [Timestamp('2024-06-05 11:59:43.706454016'),
   Timestamp('2024-06-05 11:59:45.705260032')]],
 [[Timestamp('2024-06-04 18:34:58.318092800'),
   Timestamp('2024-06-04 18:35:00.416941056')],
  [Timestamp('2024-06-04 18:38:16.373419008'),
   Timestamp('2024-06-04 18:38:19.391865856')]],
 [[Timestamp('2024-06-04 18:32:56.440315136'),
   T

In [155]:
len(extracted_eye_data[0][0]['Timestamp'])

78

In [130]:
motion_data['Timestamp'][9719]

Timestamp('2024-06-04 18:34:53.860209920')

In [ ]:
# 指定された範囲のデータをフィルタリング
circle_start_time = '2024-06-04 16:00:31'
circle_end_time = '2024-06-04 16:00:34'
cross_start_time = '2024-06-04 16:10:55'
cross_end_time = '2024-06-04 16:10:58'
tri_start_time = '2024-06-04 16:31:37'
tri_end_time = '2024-06-04 16:31:40'
check_start_time = '2024-06-04 16:46:43'
check_end_time = '2024-06-04 16:46:46'
circle = motion_data[(motion_data['Timestamp'] >= circle_start_time) & (motion_data['Timestamp'] <= circle_end_time)].reset_index(drop=True)
cross = motion_data[(motion_data['Timestamp'] >= cross_start_time) & (motion_data['Timestamp'] <= cross_end_time)].reset_index(drop=True)
tri = motion_data[(motion_data['Timestamp'] >= tri_start_time) & (motion_data['Timestamp'] <= tri_end_time)].reset_index(drop=True)
check = motion_data[(motion_data['Timestamp'] >= check_start_time) & (motion_data['Timestamp'] <= check_end_time)].reset_index(drop=True)

In [ ]:
Th = 3.5

In [ ]:
segx = mr.spring_ogawa(motion_data['AccelerationX'], circle['AccelerationX'], Th)
segy = mr.spring_ogawa(motion_data['AccelerationY'], circle['AccelerationY'], Th)
segz = mr.spring_ogawa(motion_data['AccelerationZ'], circle['AccelerationZ'], Th)

In [ ]:
overlap_ranges = mr.combine_and_find_overlapping_segments(segx, segy, segz)

In [ ]:
overlap_ranges = mr.combine_and_find_all_overlapping_segments(segx, segy, segz)

In [ ]:
circle35 = overlap_ranges

In [ ]:
motion_data['Timestamp'][9760]

In [ ]:
cross3 = overlap_ranges

In [ ]:
tri35[36]

In [ ]:
check35

In [ ]:
motion_data['Timestamp'][66190]

In [ ]:
gesture_eye_data = []

In [ ]:
results = []
for start, end in circle35:
    a = [motion_data['Timestamp'][start], motion_data['Timestamp'][end]]
    results.append(a)

In [ ]:
# 経過時間を計算して格納するリスト
elapsed_times = []

# 各区間の経過時間を計算
for start_time, end_time in results:
    # タイムスタンプをdatetimeオブジェクトに変換
    start_dt = start_time
    end_dt = end_time

    # 経過時間を計算
    elapsed_time = (end_dt - start_dt).total_seconds()

    # 経過時間をリストに追加
    elapsed_times.append(elapsed_time)

# 経過時間のリストを表示
print("Elapsed times (seconds):", elapsed_times)

In [ ]:
gesture_eye_data = []

In [ ]:
for i in range(len(results)):
  gesture_eye_data.append(eye_data[(eye_data['Timestamp'] >= (results[i][0] - pd.Timedelta(seconds=5))) & (eye_data['Timestamp'] <= (results[i][1]) + pd.Timedelta(seconds=5))])

In [ ]:
len(gesture_eye_data)

In [ ]:
plt.figure(figsize=(72, 6))
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
plt.grid(True)  # グリッド表示
plt.legend()  # 凡例表示
plt.xticks(rotation=45)  # x軸のラベルを45度回転
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
# ハイライト
plt.axvspan(results[36][0], results[36][1], color='yellow', alpha=0.3)
plt.show()

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()

In [ ]:
def calculate_variance(data, start_time, end_time):
    # 指定された期間内のデータをフィルタリング
    filtered_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    # 差分の分散を計算
    gaze_x_var = filtered_data['Gaze point X'].diff().var()
    gaze_y_var = filtered_data['Gaze point Y'].diff().var()
    return gaze_x_var, gaze_y_var


In [ ]:
# 分散を格納する配列
variances_x = []
variances_y = []
# グラフのプロットと分散の計算
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))

    # ハイライト区間の開始と終了時刻
    start_time = results[i][0]
    end_time = results[i][1]

    # ハイライト
    plt.axvspan(start_time, end_time, color='yellow', alpha=0.3)

    # 分散の計算
    gaze_x_var, gaze_y_var = calculate_variance(gesture_eye_data[i], start_time, end_time)

    # 分散を配列に格納
    variances_x.append(gaze_x_var)
    variances_y.append(gaze_y_var)

    # 分散の表示
    plt.title(f'Gaze point X variance: {gaze_x_var:.5f}, Gaze point Y variance: {gaze_y_var:.5f}')

    plt.show()

# 分散の配列を表示
print("Gaze point X variances:", variances_x)
print("Gaze point Y variances:", variances_y)

In [ ]:
np.sqrt(variances_x)

In [ ]:
np.sqrt(variances_y)

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()